# Projet Walmart sales

Bloc3 : PROJECTS Supervised Machine Learning

Walmart sales

360 min

https://app.jedha.co/course/projects-supervised-machine-learning-ft/walmart-sales-ft

## Company's Description 📇

Walmart Inc. is an American multinational retail corporation that operates a chain of hypermarkets, discount department stores, and grocery stores from the United States, headquartered in Bentonville, Arkansas. The company was founded by Sam Walton in 1962.

## Project 🚧

Walmart's marketing service has asked you to build a machine learning model able to estimate the weekly sales in their stores, with the best precision possible on the predictions made. Such a model would help them understand better how the sales are influenced by economic indicators, and might be used to plan future marketing campaigns.

## Goals 🎯

The project can be divided into three steps:

- Part 1 : make an EDA and all the necessary preprocessings to prepare data for machine learning
- Part 2 : train a **linear regression model** (baseline)
- Part 3 : avoid overfitting by training a **regularized regression model**

In [873]:
#import libraries for EDA
import pandas as pd
import numpy as np

# Visualization libraries
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


#from src.eda import *
# import libraries for modeling
# preprocessing selection
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
# model selection
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import cross_val_score, GridSearchCV
# model evaluation

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score



In [874]:

# For reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

In [875]:
def how_null_is_it(df: pd.DataFrame):
        print()
        print(f"Overall missing values in dataset : {df.isnull().sum().sum()}")
        print( )
        print("missing values in dataset per column :")
        print(df.isnull().sum() )
        print( )

def summary(df: pd.DataFrame):
        """Print a summary of the dataset."""
        print("________________________________________________" )
        print("Data Start")
        display(df.head(10) )
        print()
        print("Data End")
        display(df.tail(10) )
        print()
        print("shape of the dataset : ")
        display(df.shape)
        print()
        print("columns of the dataset : ")
        display(df.columns)
        print()
        print("data describe : ")
        display(df.describe(include='all') )
        print()
        print("types in dataset :")
        display(df.dtypes)
        print()
        print(f"Overall missing values in dataset : {df.isnull().sum().sum()}")
        print( )
        print("missing values in dataset per column :")
        display(df.isnull().sum() )
        print("________________________________________________" )
def score_model(model,x_train, y_train, x_test, y_test):
    print(model.score(x_train, y_train))
    print(model.score(x_test, y_test))

## Part 0 : import dataset and inception

We import the csv dataset into a pandas dataframe and have a first look to the datastructure.
Since we know that original data come from Kaggle before modifying by Jedha, we watch in Kaggle about the meaning of eaxh column. 
This is the historical data that covers sales from 2010-02-05 to 2012-11-01, in the file Walmart_Store_sales. Within this file you will find the following fields:


 |  column name   |   description   |  
 |  -------- | ------- |
 | Store | the store number | 
 | Date | the week of sales | 
 | Weekly_Sales  |  sales for the given store | 
 |  Holiday_Flag  | whether the week is a special holiday week <br /> 1 – Holiday week <br /> 0 – Non-holiday week | 
 | Temperature |  Temperature on the day of sale | 
 | Fuel_Price  |  Cost of fuel in the region | 
 | CPI |  Prevailing consumer price index  | 
 | Unemployment  |  Prevailing unemployment rate  | 
 | Holiday Events  |    Super Bowl: 12-Feb-10, 11-Feb-11, 10-Feb-12, 8-Feb-13<br /> Labour Day: 10-Sep-10, 9-Sep-11, 7-Sep-12, 6-Sep-13<br /> Thanksgiving: 26-Nov-10, 25-Nov-11, 23-Nov-12, 29-Nov-13<br /> Christmas: 31-Dec-10, 30-Dec-11, 28-Dec-12, 27-Dec-13  | 

 L'indice des prix à la consommation ou IPC (en anglais, consumer price index ou CPI) mesure l'évolution du niveau moyen des prix des biens et services consommés par les ménages, pondérés par leur part dans la consommation moyenne des ménages. L'indice (105 par exemple) permet de mesurer l'inflation (ici +5 % de hausse des prix), ou la déflation en cas de baisse des prix, sur une période et donc l'évolution de la valeur de la monnaie (la valeur de la monnaie diminue lorsque les prix augmentent). Le taux (annuel) d'inflation désigne généralement, lorsque l'indice n'est pas précisé, le pourcentage d'augmentation de cet indice (IPC) particulier sur une année.

 

In [876]:
df= pd.read_csv("Data/Walmart_Store_sales.csv")
summary(df)

________________________________________________
Data Start


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092
5,4.0,28-05-2010,1857533.70,0.0,NaN,2.756,126.160226,7.896
6,15.0,03-06-2011,695396.19,0.0,69.80,4.069,134.855161,7.658
7,20.0,03-02-2012,2203523.20,0.0,39.93,3.617,213.023622,6.961
8,14.0,10-12-2010,2600519.26,0.0,30.54,3.109,NaN,NaN
9,3.0,NaN,418925.47,0.0,60.12,3.555,224.132020,6.833



Data End


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
140,3.0,07-01-2011,NaN,0.0,53.35,2.976,214.695510,7.551
141,5.0,12-11-2010,301827.36,0.0,62.37,NaN,212.560411,6.768
142,3.0,07-10-2011,403342.40,0.0,75.54,3.285,NaN,7.197
143,3.0,04-06-2010,396968.80,0.0,78.53,2.705,214.495838,7.343
144,3.0,19-10-2012,424513.08,0.0,73.44,3.594,226.968844,6.034
145,14.0,18-06-2010,2248645.59,0.0,72.62,2.780,182.442420,8.899
146,7.0,NaN,716388.81,NaN,20.74,2.778,NaN,NaN
147,17.0,11-06-2010,845252.21,0.0,57.14,2.841,126.111903,NaN
148,8.0,12-08-2011,856796.10,0.0,86.05,3.638,219.007525,NaN
149,19.0,20-04-2012,1255087.26,0.0,55.20,4.170,137.923067,8.150



shape of the dataset : 


...


types in dataset :


Store           float64
Date             object
Weekly_Sales    float64
Holiday_Flag    float64
Temperature     float64
Fuel_Price      float64
CPI             float64
Unemployment    float64
dtype: object


Overall missing values in dataset : 103

missing values in dataset per column :


Store            0
Date            18
Weekly_Sales    14
Holiday_Flag    12
Temperature     18
Fuel_Price      14
CPI             12
Unemployment    15
dtype: int64

________________________________________________


In [877]:
display(df)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092
...,...,...,...,...,...,...,...,...
145,14.0,18-06-2010,2248645.59,0.0,72.62,2.780,182.442420,8.899
146,7.0,NaN,716388.81,NaN,20.74,2.778,NaN,NaN
147,17.0,11-06-2010,845252.21,0.0,57.14,2.841,126.111903,NaN
148,8.0,12-08-2011,856796.10,0.0,86.05,3.638,219.007525,NaN


We noticed also that the Store is an integer id to identify the store. We convert it into intger since the store sounds more like a classification than a regression value.


In [878]:
df=df.dropna()
df.reset_index(drop=True)
df.shape


(75, 8)

In [879]:
df = df.astype({"Store": int, "Weekly_Sales": float, "Temperature": float, "Fuel_Price": float, "CPI": float, "Unemployment": float},errors='ignore')

In [880]:
df['Holiday_Flag'] = df['Holiday_Flag'].astype('int64')
df.dtypes
df

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
1,13,25-03-2011,1807545.43,0,42.38,3.435,128.616064,7.470
4,6,28-05-2010,1644470.66,0,78.89,2.759,212.412888,7.092
6,15,03-06-2011,695396.19,0,69.80,4.069,134.855161,7.658
7,20,03-02-2012,2203523.20,0,39.93,3.617,213.023622,6.961
10,8,19-08-2011,895066.50,0,82.92,3.554,219.070197,6.425
...,...,...,...,...,...,...,...,...
139,7,25-05-2012,532739.77,0,50.60,3.804,197.588605,8.090
143,3,04-06-2010,396968.80,0,78.53,2.705,214.495838,7.343
144,3,19-10-2012,424513.08,0,73.44,3.594,226.968844,6.034
145,14,18-06-2010,2248645.59,0,72.62,2.780,182.442420,8.899


The date is an object time, we need to convert in real date figures (new column dt) from the string format _day_-_month_-_year_ (`"%d-%m-%Y"`)

In [881]:
df['dt'] = pd.to_datetime(df["Date"], format = "%d-%m-%Y")
df.dtypes

Store                    int64
Date                    object
Weekly_Sales           float64
Holiday_Flag             int64
Temperature            float64
Fuel_Price             float64
CPI                    float64
Unemployment           float64
dt              datetime64[ns]
dtype: object

We split the date on year, month and week.

In [882]:
df['Year'] = df['dt'].dt.year.astype('Int64')
df['Month'] = df['dt'].dt.month.astype('Int64')
df['Day'] = df['dt'].dt.day.astype('Int64')
df['Week'] = df['dt'].dt.isocalendar().week.astype('Int64')
df['DayOfWeek'] = df['dt'].dt.weekday.astype('Int64')

df.dtypes



Store                    int64
Date                    object
Weekly_Sales           float64
Holiday_Flag             int64
Temperature            float64
Fuel_Price             float64
CPI                    float64
Unemployment           float64
dt              datetime64[ns]
Year                     Int64
Month                    Int64
Day                      Int64
Week                     Int64
DayOfWeek                Int64
dtype: object

In [883]:
display(df)
print()
display(df.dtypes)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,dt,Year,Month,Day,Week,DayOfWeek
1,13,25-03-2011,1807545.43,0,42.38,3.435,128.616064,7.470,2011-03-25,2011,3,25,12,4
4,6,28-05-2010,1644470.66,0,78.89,2.759,212.412888,7.092,2010-05-28,2010,5,28,21,4
6,15,03-06-2011,695396.19,0,69.80,4.069,134.855161,7.658,2011-06-03,2011,6,3,22,4
7,20,03-02-2012,2203523.20,0,39.93,3.617,213.023622,6.961,2012-02-03,2012,2,3,5,4
10,8,19-08-2011,895066.50,0,82.92,3.554,219.070197,6.425,2011-08-19,2011,8,19,33,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,7,25-05-2012,532739.77,0,50.60,3.804,197.588605,8.090,2012-05-25,2012,5,25,21,4
143,3,04-06-2010,396968.80,0,78.53,2.705,214.495838,7.343,2010-06-04,2010,6,4,22,4
144,3,19-10-2012,424513.08,0,73.44,3.594,226.968844,6.034,2012-10-19,2012,10,19,42,4
145,14,18-06-2010,2248645.59,0,72.62,2.780,182.442420,8.899,2010-06-18,2010,6,18,24,4


Store                    int64
Date                    object
Weekly_Sales           float64
Holiday_Flag             int64
Temperature            float64
Fuel_Price             float64
CPI                    float64
Unemployment           float64
dt              datetime64[ns]
Year                     Int64
Month                    Int64
Day                      Int64
Week                     Int64
DayOfWeek                Int64
dtype: object

## Part 1 : Exploration, Exploratory data analysis (EDA)



In [884]:
# Constant for the layout of the plots
WIDTH = 600
HEIGHT = 400
MARGIN = 30

In [885]:
# Compute total revenue cumulated by day
df_store = df.groupby('Store')['Week'].count().reset_index()
df_store.sort_values(by='Store',ascending= False)
fig = px.pie(df_store, names = "Store", values = "Week")
fig.show()

In [886]:
orders_store_by_weekly_sales=df.groupby(['Store'])[['Weekly_Sales']].mean()
orders_store_by_weekly_sales.sort_values(by='Weekly_Sales',ascending= False)
orders_year_by_weekly_sales=df.groupby(['Year'])[['Weekly_Sales']].mean()
orders_month_by_weekly_sales=df.groupby(['Month'])[['Weekly_Sales']].mean()
orders_week_by_weekly_sales=df.groupby(['Week'])[['Weekly_Sales']].mean()
orders_week_by_weekly_sales.sort_values(by='Weekly_Sales',ascending= False)

#px.bar(orders_year_by_weekly_sales, x= orders_year_by_weekly_sales.index, y='Weekly_Sales', 
#        title="Average Weekly Sales by Year", labels={"x":"Store","Weekly_Sales":"Average Weekly Sales per Year"}).show()
#px.bar(orders_store_by_weekly_sales, x= orders_store_by_weekly_sales.index, y='Weekly_Sales', title="Average Weekly Sales by Store", labels={"x":"Store","Weekly_Sales":"Average Weekly Sales per store"}).show()
#px.bar(orders_month_by_weekly_sales, x= orders_month_by_weekly_sales.index, y='Weekly_Sales', title="Average Weekly Sales by Month", labels={"x":"Month","Weekly_Sales":"Average Weekly Sales per month"}).show()
#px.bar(orders_week_by_weekly_sales, x= orders_week_by_weekly_sales.index, y='Weekly_Sales', title="Average Weekly Sales", labels={"x":"Month","Weekly_Sales":"Average Weekly Sales"}).show()

fig = make_subplots(rows = 4, cols = 1, subplot_titles = (["Wallmart average weekly sales per store",
                                        "Wallmart average weekly sales per year",
                                        "Wallmart average weekly sales per month",
                                        "Wallmart average weekly sales per week"] ))
fig.add_bar(

        x = orders_store_by_weekly_sales.index,
        y = orders_store_by_weekly_sales.Weekly_Sales,
        row = 1,
        col = 1

)    
fig.add_bar(
        x = orders_year_by_weekly_sales.index,
        y = orders_year_by_weekly_sales['Weekly_Sales'],
        row = 2,
        col = 1

)    
fig.add_bar(

        x = orders_month_by_weekly_sales.index,
        y = orders_month_by_weekly_sales['Weekly_Sales'],
        row = 3,
        col = 1
)    
fig.add_bar(
        x = orders_week_by_weekly_sales.index,
        y = orders_week_by_weekly_sales['Weekly_Sales'],
        row = 4,
        col = 1
)    
layout = go.Layout(
    title = go.layout.Title(text = "Average Weekly Sales", x = 1.0),
    showlegend = False,
    autosize=False,
    width=1000,
    height=2000,
    xaxis=go.layout.XAxis(linecolor="black", linewidth=1, mirror=True),
    yaxis=go.layout.YAxis(linecolor="black", linewidth=1, mirror=True),
    margin=go.layout.Margin(l=50, r=50, b=100, t=100, pad=4),
)

fig.update_layout(layout)
#px.bar(orders_store_by_weekly_sales, x= orders_store_by_weekly_sales.index, y='Weekly_Sales', title="Average Weekly Sales by Store", labels={"x":"Store","Weekly_Sales":"Average Weekly Sales"}).show()

In [887]:
orders_month_by_weekly_sales=df.groupby(['Month'])[['Weekly_Sales']].sum()
orders_month_by_weekly_sales.sort_values(by='Weekly_Sales',ascending= False)
fig = px.line(orders_month_by_weekly_sales, x=orders_month_by_weekly_sales.index, y='Weekly_Sales', title="Total Weekly Sales by Month", width=WIDTH, height=HEIGHT)
fig = px.scatter(orders_month_by_weekly_sales, x=orders_month_by_weekly_sales.index, y='Weekly_Sales', title="Total Weekly Sales by Month", width=WIDTH, height=HEIGHT)
#fig.update_traces(marker=dict(size=12, color='LightSkyBlue', line=dict(width=2, color='DarkSlateGrey')), selector=dict(mode='markers'))
fig.update_layout(margin=dict(l=MARGIN, r=MARGIN, t=MARGIN, b=MARGIN))
px.density_heatmap(orders_month_by_weekly_sales, x=orders_month_by_weekly_sales.index, y='Weekly_Sales', nbinsx=20, nbinsy=10, width=WIDTH, height=HEIGHT)
px.box(df, x='Month', y='Weekly_Sales', width=WIDTH, height=HEIGHT)
fig = px.line(orders_month_by_weekly_sales, x=orders_month_by_weekly_sales.index, y='Weekly_Sales', title="Total Weekly Sales by Month", width=WIDTH, height=HEIGHT)
fig.show()

In [888]:
orders_month_by_weekly_sales

,Weekly_Sales
Month,
1,1758050.79
2,9747718.72
3,11250266.88
4,5631856.38
5,7670030.46
6,10141497.56
7,11887829.82
8,5876256.19
9,4550383.33


In [889]:
orders_week_by_weekly_sales=df.groupby(['Week'])[['Weekly_Sales']].sum()
orders_week_by_weekly_sales.sort_values(by='Weekly_Sales',ascending= False)
fig = px.line(orders_week_by_weekly_sales, x=orders_week_by_weekly_sales.index, y='Weekly_Sales', title="Total Weekly Sales by week", width=WIDTH, height=HEIGHT)
fig = px.scatter(orders_week_by_weekly_sales, x=orders_week_by_weekly_sales.index, y='Weekly_Sales', title="Total Weekly Sales by week", width=WIDTH, height=HEIGHT)
#fig.update_traces(marker=dict(size=12, color='LightSkyBlue', line=dict(width=2, color='DarkSlateGrey')), selector=dict(mode='markers'))
fig.update_layout(margin=dict(l=MARGIN, r=MARGIN, t=MARGIN, b=MARGIN))
px.density_heatmap(orders_week_by_weekly_sales, x=orders_week_by_weekly_sales.index, y='Weekly_Sales', nbinsx=20, nbinsy=10, width=WIDTH, height=HEIGHT)
px.box(df, x='Week', y='Weekly_Sales', width=WIDTH, height=HEIGHT)
fig = px.line(orders_week_by_weekly_sales, x=orders_week_by_weekly_sales.index, y='Weekly_Sales', title="Total Weekly Sales by Week", width=WIDTH, height=HEIGHT)
fig.show()

In [890]:
orders_week_by_weekly_sales

,Weekly_Sales
Week,
1,1758050.79
5,2665145.42
6,3200219.61
7,2020550.99
8,1861802.70
9,1990371.02
11,4273666.63
12,4501133.82
13,1046240.55


In [891]:
fig = px.line(orders_week_by_weekly_sales, x=orders_week_by_weekly_sales.index, y='Weekly_Sales' , title='Weekly Sales according to week', width=WIDTH, height=HEIGHT)
fig.update_layout(margin=dict(l=MARGIN, r=MARGIN, t=MARGIN, b=MARGIN))

In [892]:
fig = px.line(orders_month_by_weekly_sales, x=orders_month_by_weekly_sales.index, y='Weekly_Sales' , title='Weekly Sales according to month', width=WIDTH, height=HEIGHT)
fig.update_layout(margin=dict(l=MARGIN, r=MARGIN, t=MARGIN, b=MARGIN))

In [893]:
dfcpi=df[df['CPI'].notna()]
cpi_over_store = dfcpi.groupby('Store')['CPI'].mean()
display(cpi_over_store)

Store
1     215.252979
2     214.333910
3     220.325272
4     128.727135
5     217.419409
6     214.713791
7     195.476131
8     215.876613
9     215.437285
10    130.682713
11    215.061403
12    127.810020
13    128.864857
14    186.141628
15    136.219774
16    193.255923
17    127.968344
18    135.566845
19    134.483065
20    207.411465
Name: CPI, dtype: float64

In [894]:
px.density_heatmap(df, x='Store', y='Weekly_Sales', nbinsx=20, nbinsy=40, width=WIDTH, height=HEIGHT)

In [895]:
orders_month_by_weekly_sales=df.groupby(['Year','Month'])['Weekly_Sales'].sum()
display(orders_month_by_weekly_sales)

Year  Month
2010  2        3098381.06
      3        1427023.45
      4        1318883.90
      5        3138722.16
      6        4982827.11
      7        5816911.71
      8        2054934.63
      9        1739826.88
      10       1858825.37
      11       4193878.55
      12       7859316.35
2011  1        1758050.79
      3        3074110.37
      4        1639358.93
      5        2161575.73
      6        2703741.11
      7        4264918.32
      8        3821321.56
      9        2212679.90
      11       2250229.37
      12       2771397.17
2012  2        6649337.66
      3        6749133.06
      4        2673613.55
      5        2369732.57
      6        2454929.34
      7        1805999.79
      9         597876.55
      10       2731828.57
Name: Weekly_Sales, dtype: float64

In [896]:
orders_month_by_weekly_sales.index

MultiIndex([(2010,  2),
            (2010,  3),
            (2010,  4),
            (2010,  5),
            (2010,  6),
            (2010,  7),
            (2010,  8),
            (2010,  9),
            (2010, 10),
            (2010, 11),
            (2010, 12),
            (2011,  1),
            (2011,  3),
            (2011,  4),
            (2011,  5),
            (2011,  6),
            (2011,  7),
            (2011,  8),
            (2011,  9),
            (2011, 11),
            (2011, 12),
            (2012,  2),
            (2012,  3),
            (2012,  4),
            (2012,  5),
            (2012,  6),
            (2012,  7),
            (2012,  9),
            (2012, 10)],
           names=['Year', 'Month'])

In [897]:
year=2010
orders_month_by_weekly_sales.loc[year]

Month
2     3098381.06
3     1427023.45
4     1318883.90
5     3138722.16
6     4982827.11
7     5816911.71
8     2054934.63
9     1739826.88
10    1858825.37
11    4193878.55
12    7859316.35
Name: Weekly_Sales, dtype: float64

In [898]:
orders_month_by_weekly_sales=df.groupby(['Year','Month'])['Weekly_Sales'].sum()
dfm=orders_month_by_weekly_sales
years = [2010, 2011, 2012]

fig = go.Figure()

visible = True, False, False, False, False

for i, year in enumerate(years):
    if i == 0:
        fig.add_trace(
            go.Scatter(
                x=dfm[year].index,
                y=dfm,
                visible=True
            )
        )

    else:
        fig.add_trace(
            go.Scatter(
                x=dfm[year].index,
                y=dfm,
                visible=False
            )
        )
        


fig.update_layout(
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=[
            go.layout.updatemenu.Button(
                label="2010",
                method='update',
                args=[{'visible': [True,  False, False],
                        "xaxis": dict(range=[1, 12], title="Month", tick0=1,dtick=1)
                }]
            ),
            go.layout.updatemenu.Button(
                label="2011",
                method='update',
                args=[{'visible': [False,  True,  False],
                        "xaxis": dict(range=[1, 12], title="Month", tick0=1,dtick=1)
                }]
            ),
            go.layout.updatemenu.Button(
                label="2012",
                method='update',
                args=[{'visible': [False, False,  True],
                        "xaxis": dict(range=[1, 12], title="Month", tick0=1,dtick=1)
                }]
            ),
            
        ]
    )]
)

fig.update_layout(title=dict(text="Monthly Sales observations in a chosen year", x=0.5))

fig.show()

In [899]:
orders_week_by_weekly_sales=df.groupby(['Year','Week'])['Weekly_Sales'].sum()
dfw=orders_week_by_weekly_sales
years = [2010, 2011, 2012]

fig = go.Figure()

visible = True, False, False, False, False

for i, year in enumerate(years):
    if i == 0:
        fig.add_trace(
            go.Scatter(
                x=dfw[year].index,
                y=dfw,
                visible=True
            )
        )

    else:
        fig.add_trace(
            go.Scatter(
                x=dfw[year].index,
                y=dfw,
                visible=False
            )
        )
        


fig.update_layout(
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=[
            go.layout.updatemenu.Button(
                label="2010",
                method='update',
                args=[{'visible': [True,  False, False],
                        "xaxis": dict(range=[1, 52], title="Week", tick0=1,dtick=4)
                }]
            ),
            go.layout.updatemenu.Button(
                label="2011",
                method='update',
                args=[{'visible': [False,  True,  False],
                        "xaxis": dict(range=[1, 52], title="Week", tick0=1,dtick=4)
                }]
            ),
            go.layout.updatemenu.Button(
                label="2012",
                method='update',
                args=[{'visible': [False, False,  True],
                       "xaxis": dict(range=[1, 52], title="Week", tick0=1,dtick=4)
                }]
            ),
            
        ]
    )]
)

fig.update_layout(title=dict(text="Weekly Sales observations in a chosen year", x=0.5))

fig.show()

In [900]:
df.describe()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,dt,Year,Month,Day,Week,DayOfWeek
count,75.000000,7.500000e+01,75.000000,75.000000,75.000000,75.000000,75.000000,75,75.0,75.0,75.0,75.0,75.0
mean,10.093333,1.202392e+06,0.093333,60.469600,3.325920,176.073438,7.660453,2011-05-29 03:31:12,2010.906667,6.533333,15.586667,26.16,4.0
min,1.000000,2.689290e+05,0.000000,18.790000,2.548000,126.114581,5.143000,2010-02-05 00:00:00,2010.0,1.0,1.0,1.0,4.0
25%,4.500000,5.469425e+05,0.000000,45.465000,2.835000,131.033780,6.597500,2010-09-03 00:00:00,2010.0,4.0,10.0,16.0,4.0
50%,10.000000,1.074079e+06,0.000000,61.110000,3.435000,192.826069,7.470000,2011-05-20 00:00:00,2011.0,6.0,16.0,25.0,4.0
75%,15.500000,1.806773e+06,0.000000,75.180000,3.742000,214.703622,8.102500,2012-02-27 12:00:00,2012.0,9.0,22.0,37.5,4.0
max,20.000000,2.771397e+06,1.000000,91.040000,4.193000,226.968844,14.313000,2012-10-19 00:00:00,2012.0,12.0,31.0,52.0,4.0
std,6.100893,6.754152e+05,0.292858,17.338982,0.494012,40.155442,1.763908,NaN,0.841063,3.120695,8.332764,13.625334,0.0


In [901]:
df_c=df.drop(columns=['dt','Date'])
# Correlation
df_corr = df_c.corr().round(1)  
# Mask to matrix
mask = np.zeros_like(df_corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
# Viz
df_corr_viz = df_corr.mask(mask).dropna(how='all').dropna( how='all')
fig = px.imshow(df_corr_viz, text_auto=True)
fig.show()

In [902]:
df_corr

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,Week,DayOfWeek
Store,1.0,0.2,-0.0,-0.2,0.2,-0.6,0.2,-0.0,-0.0,-0.0,-0.0,NaN
Weekly_Sales,0.2,1.0,-0.0,-0.1,-0.1,-0.4,-0.0,-0.1,0.0,-0.1,0.0,NaN
Holiday_Flag,-0.0,-0.0,1.0,-0.2,-0.2,0.2,0.2,-0.1,0.0,-0.0,0.0,NaN
Temperature,-0.2,-0.1,-0.2,1.0,-0.0,0.1,0.0,-0.1,0.2,0.1,0.2,NaN
Fuel_Price,0.2,-0.1,-0.2,-0.0,1.0,-0.2,0.0,0.8,-0.2,0.0,-0.2,NaN
CPI,-0.6,-0.4,0.2,0.1,-0.2,1.0,-0.3,0.0,-0.1,0.2,-0.1,NaN
Unemployment,0.2,-0.0,0.2,0.0,0.0,-0.3,1.0,-0.2,0.0,-0.2,0.0,NaN
Year,-0.0,-0.1,-0.1,-0.1,0.8,0.0,-0.2,1.0,-0.3,-0.0,-0.3,NaN
Month,-0.0,0.0,0.0,0.2,-0.2,-0.1,0.0,-0.3,1.0,0.0,1.0,NaN
Day,-0.0,-0.1,-0.0,0.1,0.0,0.2,-0.2,-0.0,0.0,1.0,0.1,NaN


In [903]:
px.imshow(df_c.corr())

# 0.0 - 0.3 = No correlation
# 0.3 - 0.5 = Weak correlation
# 0.5 - 0.7 = Moderate correlation
# 0.7 - 1.0 = Strong correlation

In [904]:
Sales_per_week=df.groupby(['Year','Week'])[['Weekly_Sales','CPI',"Temperature"]].sum()
df_spw=Sales_per_week

In [905]:
px.scatter_matrix(df, width=1000, height=1000)

## Preprocessing - pandas part 🐼🐼 
In this dataset, some features are removed since they are useless for the modelling.

### Remove row where target values are missing

Drop lines where target values are missing :

Here, the target variable (y) corresponds to the column Weekly_Sales. One can see above that there are some missing values in this column.
We never use imputation techniques on the target : it might create some bias in the predictions !
Then, we will just drop the lines in the dataset for which the value in Weekly_Sales is missing.

We noticed that 14 Weekly_Sales are missing in the dataset.

Since this is the target value we have no other choice than remove thes rows from the original dataset.



In [906]:
df_model = df[df['Weekly_Sales'].notna()]
print(df_model['Weekly_Sales'].isnull().sum())
df_model.describe()

0


,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,dt,Year,Month,Day,Week,DayOfWeek
count,75.000000,7.500000e+01,75.000000,75.000000,75.000000,75.000000,75.000000,75,75.0,75.0,75.0,75.0,75.0
mean,10.093333,1.202392e+06,0.093333,60.469600,3.325920,176.073438,7.660453,2011-05-29 03:31:12,2010.906667,6.533333,15.586667,26.16,4.0
min,1.000000,2.689290e+05,0.000000,18.790000,2.548000,126.114581,5.143000,2010-02-05 00:00:00,2010.0,1.0,1.0,1.0,4.0
25%,4.500000,5.469425e+05,0.000000,45.465000,2.835000,131.033780,6.597500,2010-09-03 00:00:00,2010.0,4.0,10.0,16.0,4.0
50%,10.000000,1.074079e+06,0.000000,61.110000,3.435000,192.826069,7.470000,2011-05-20 00:00:00,2011.0,6.0,16.0,25.0,4.0
75%,15.500000,1.806773e+06,0.000000,75.180000,3.742000,214.703622,8.102500,2012-02-27 12:00:00,2012.0,9.0,22.0,37.5,4.0
max,20.000000,2.771397e+06,1.000000,91.040000,4.193000,226.968844,14.313000,2012-10-19 00:00:00,2012.0,12.0,31.0,52.0,4.0
std,6.100893,6.754152e+05,0.292858,17.338982,0.494012,40.155442,1.763908,NaN,0.841063,3.120695,8.332764,13.625334,0.0


### Remove duplicate rows

In [907]:

rs,cs = df.shape

df_model.drop_duplicates(inplace=True)

if df_model.shape==(rs,cs):
    print('\n\033[1mInference:\033[0m The dataset doesn\'t have any duplicates')
else:
    print(f'\n\033[1mInference:\033[0m Number of duplicates dropped/fixed ---> {rs-df_model.shape[0]}')


Inference: The dataset doesn't have any duplicates


### Remove row with Not Available data

In [908]:
df_model=df_model.dropna()
df_model.reset_index(drop=True)
df_model.shape


(75, 14)

### Remove Date with date format and wrong format


Remove row with Not available Date.

In [909]:
#df=df.dropna(subset=['Date'])

In [910]:
df_model=df_model.drop('Date', axis=1)
df_model=df_model.drop('dt', axis=1)
df_model.reset_index(drop=True)

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,Week,DayOfWeek
0,13,1807545.43,0,42.38,3.435,128.616064,7.470,2011,3,25,12,4
1,6,1644470.66,0,78.89,2.759,212.412888,7.092,2010,5,28,21,4
2,15,695396.19,0,69.80,4.069,134.855161,7.658,2011,6,3,22,4
3,20,2203523.20,0,39.93,3.617,213.023622,6.961,2012,2,3,5,4
4,8,895066.50,0,82.92,3.554,219.070197,6.425,2011,8,19,33,4
...,...,...,...,...,...,...,...,...,...,...,...,...
70,7,532739.77,0,50.60,3.804,197.588605,8.090,2012,5,25,21,4
71,3,396968.80,0,78.53,2.705,214.495838,7.343,2010,6,4,22,4
72,3,424513.08,0,73.44,3.594,226.968844,6.034,2012,10,19,42,4
73,14,2248645.59,0,72.62,2.780,182.442420,8.899,2010,6,18,24,4


### Holidays Flag analysis

There is a very small correlation between Weekly_Sales and Holiday_Flag.

In [911]:
print(f"Moreover there is only  {df_model['Holiday_Flag'].sum()}  holidays rows  in the dataset " )
print(f"and {df_model['Holiday_Flag'].isna().sum()} Non available values over {df_model.shape[0]} rows.") 

Moreover there is only  7  holidays rows  in the dataset 
and 0 Non available values over 75 rows.


We can remove Holidays data from this dataset which is more a perturbation data than a descriptive data for the Weekly Sales prediction".

In [912]:
#df_model = df_model.drop(columns=['Holiday_Flag'])

### Fuel Price analysis

There is no correlation between Weekly_Sales and Fuel Price.

In [913]:
print(f"Moreover there is {df_model['Fuel_Price'].isna().sum()} Non available values over {df_model.shape[0]} rows.") 

Moreover there is 0 Non available values over 75 rows.


In [914]:
df_model = df_model.drop(columns=['Fuel_Price'])
how_null_is_it(df_model)


Overall missing values in dataset : 0

missing values in dataset per column :
Store           0
Weekly_Sales    0
Holiday_Flag    0
Temperature     0
CPI             0
Unemployment    0
Year            0
Month           0
Day             0
Week            0
DayOfWeek       0
dtype: int64



We can remove Fuel Price from this dataset data which is more a perturbation data than a descriptive data for the Weekly Sales prediction".

In [915]:
y=df_model['Weekly_Sales']
print(y)

1      1807545.43
4      1644470.66
6       695396.19
7      2203523.20
10      895066.50
          ...    
139     532739.77
143     396968.80
144     424513.08
145    2248645.59
149    1255087.26
Name: Weekly_Sales, Length: 75, dtype: float64


### Outlier analysis

We track outliers to remove these rows.



In [916]:
numeric_list = ['Weekly_Sales', 'Temperature', 'CPI', 'Unemployment']

def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers

for col in numeric_list:
    outliers = detect_outliers_iqr(df_model, col)
    print(f"{col} -> Outlier : {outliers.shape[0]}")

Weekly_Sales -> Outlier : 0
Temperature -> Outlier : 0
CPI -> Outlier : 0
Unemployment -> Outlier : 4


##### Unemployement outlier

In [917]:

outliers_unemp = detect_outliers_iqr(df_model, 'Unemployment')
outliers_unemp.index
#
df_model=df_model.drop(outliers_unemp.index,axis=0)
df_model.reset_index(drop=True)
display(df_model)

,Store,Weekly_Sales,Holiday_Flag,Temperature,CPI,Unemployment,Year,Month,Day,Week,DayOfWeek
1,13,1807545.43,0,42.38,128.616064,7.470,2011,3,25,12,4
4,6,1644470.66,0,78.89,212.412888,7.092,2010,5,28,21,4
6,15,695396.19,0,69.80,134.855161,7.658,2011,6,3,22,4
7,20,2203523.20,0,39.93,213.023622,6.961,2012,2,3,5,4
10,8,895066.50,0,82.92,219.070197,6.425,2011,8,19,33,4
...,...,...,...,...,...,...,...,...,...,...,...
139,7,532739.77,0,50.60,197.588605,8.090,2012,5,25,21,4
143,3,396968.80,0,78.53,214.495838,7.343,2010,6,4,22,4
144,3,424513.08,0,73.44,226.968844,6.034,2012,10,19,42,4
145,14,2248645.59,0,72.62,182.442420,8.899,2010,6,18,24,4


## Preprocessing - scikit-learn part 🔬🔬
We will use ColumnTransformer and Pipeline from sklearn to preprocess the data before modeling.

In [918]:

from sklearn.model_selection import train_test_split
x=df_model.drop('Weekly_Sales', axis=1)
y=df_model['Weekly_Sales']
x_train, x_test, y_train, y_test= train_test_split(x,y, test_size=0.2, random_state= 42)
print("Train set:", x_train.shape, y_train.shape)
print("Test set:", x_test.shape, y_test.shape)

Train set: (56, 10) (56,)
Test set: (15, 10) (15,)


In [919]:
x

,Store,Holiday_Flag,Temperature,CPI,Unemployment,Year,Month,Day,Week,DayOfWeek
1,13,0,42.38,128.616064,7.470,2011,3,25,12,4
4,6,0,78.89,212.412888,7.092,2010,5,28,21,4
6,15,0,69.80,134.855161,7.658,2011,6,3,22,4
7,20,0,39.93,213.023622,6.961,2012,2,3,5,4
10,8,0,82.92,219.070197,6.425,2011,8,19,33,4
...,...,...,...,...,...,...,...,...,...,...
139,7,0,50.60,197.588605,8.090,2012,5,25,21,4
143,3,0,78.53,214.495838,7.343,2010,6,4,22,4
144,3,0,73.44,226.968844,6.034,2012,10,19,42,4
145,14,0,72.62,182.442420,8.899,2010,6,18,24,4


Create the preprocessing pipeline for numeric columns

* list of numerical columns
* impute numeric -> median
* standardise

In [920]:
numerical_columns = x[['Store','Temperature', 'CPI', 'Unemployment', 'Year', 'Month', 'Week','Day','DayOfWeek']].columns.tolist() #x.select_dtypes(include=np.number).columns.tolist()
#numerical_columns =x.select_dtypes(exclude="object").columns
numerical_columns

['Store',
 'Temperature',
 'CPI',
 'Unemployment',
 'Year',
 'Month',
 'Week',
 'Day',
 'DayOfWeek']

In [921]:

numeric_imputer = SimpleImputer(strategy='median')
numerical_scaler = StandardScaler()
numerical_pipeline = Pipeline(steps=[
    ('num_imputer', numeric_imputer),
    ('num_scaler', numerical_scaler)
])

Create the preprocessing pipeline for category columns

In [922]:
#categorical_columns = x.select_dtypes(include="object").columns #x.select_dtypes(exclude=np.number).columns.tolist()
categorical_columns = x[['Store','Holiday_Flag']].columns.tolist()
categorical_columns

['Store', 'Holiday_Flag']

In [923]:


categorical_imputer = SimpleImputer(strategy='most_frequent')
#categorical_imputer = SimpleImputer(strategy='constant', fill_value='Unknown')

categorical_encoder = OneHotEncoder(drop='first')

categorical_pipeline = Pipeline(steps=[
    ('cat_imputer', categorical_imputer),
    ('cat_encoder', categorical_encoder)
])

In [924]:
# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_pipeline, numerical_columns),
        ("cat", categorical_pipeline, categorical_columns),
    ]
)

In [925]:
print(x_test)
print()
print(x_train)

     Store  Holiday_Flag  Temperature         CPI  Unemployment  Year  Month  \
40      18             0        50.43  137.978133         8.304  2012      4   
1       13             0        42.38  128.616064         7.470  2011      3   
100      3             0        83.52  214.785826         7.343  2010      6   
10       8             0        82.92  219.070197         6.425  2011      8   
114     11             1        69.90  215.061403         7.564  2010     11   
35      19             0        33.26  133.958742         7.771  2011      3   
21       5             0        77.38  216.534361         6.489  2011      5   
59      14             0        36.85  189.842483         8.424  2012      2   
94      14             0        78.47  186.139981         8.625  2011      7   
23      19             0        39.07  131.863129         8.350  2010      3   
55      13             0        63.60  129.518333         6.877  2011      9   
20       7             0        38.26  1

In [926]:
# Preprocessings on train set
print("Performing preprocessings on train set...")
print(x_train.head())
x_train = preprocessor.fit_transform(x_train)
print("...Done.")
print(
    x_train[0:5]
)  # MUST use this syntax because X_train is a numpy array and not a pandas DataFrame anymore
print()

# Preprocessings on test set
print("Performing preprocessings on test set...")
print(x_test.head())
x_test = preprocessor.transform(x_test)  # Don't fit again !! The test set is used for validating decisions
# we made based on the training set, therefore we can only apply transformations that were parametered using the training set.
# Otherwise this creates what is called a leak from the test set which will introduce a bias in all your results.
print("...Done.")
print(
    x_test[0:5, :]
)  # MUST use this syntax because X_test is a numpy array and not a pandas DataFrame anymore
print()

Performing preprocessings on train set...
     Store  Holiday_Flag  Temperature         CPI  Unemployment  Year  Month  \
54      10             0        86.87  130.719633         7.170  2012      7   
122      7             1        57.84  198.095048         7.872  2012      9   
63       5             0        69.17  224.019287         5.422  2012     10   
121      5             0        89.42  216.046436         6.529  2011      7   
78       1             0        62.25  218.220509         7.866  2011     11   

     Day  Week  DayOfWeek  
54     6    27          4  
122    7    36          4  
63    19    42          4  
121   29    30          4  
78    18    46          4  
...Done.
[[ 0.10431561  1.5802326  -1.2282192  -0.04204051  1.15470054  0.11449715
   0.02878999 -1.07095438  0.          0.          0.          0.
   0.          0.          0.          0.          0.          1.
   0.          0.          0.          0.          0.          0.
   0.          0.          0

ValueError: Found unknown categories [np.int64(11)] in column 0 during transform

In [ ]:
x_train

## Model Training
We start training a baseline model, we'll analyze the results and we'll build improved model.
###  Baseline model (linear regression)
Once you've trained a first model, don't forget to assess its performances on the train and test sets. Are you satisfied with the results ?
Besides, it would be interesting to analyze the values of the model's coefficients to know what features are important for the prediction. To do so, the `.coef_` attribute of scikit-learn's LinearRegression class might be useful. Please refer to the following link for more information 😉 https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html



In [ ]:
# Model training
model = LinearRegression()
model.fit(x_train, y_train)

#### Model estimation

In [ ]:
y_pred = model.predict(x_test)



# Metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("Model Performance:")
score_model(model,x_train, y_train, x_test, y_test)
print(f"MAE  : {mae:.2f}")
print(f"MSE  : {mse:.2f}")
print(f"RMSE : {rmse:.2f}")
print(f"R²   : {r2:.2f}")

In [ ]:
print(x.columns)
print()
print(model.coef_)

In [ ]:
preprocessor.get_feature_names_out()

In [ ]:
coef_df = pd.DataFrame({
    'Feature': preprocessor.get_feature_names_out(),
    'Coefficient': model.coef_
}).sort_values(by='Coefficient', ascending=False)

print("Feature Importance (Linear Regression Coefficients):")
print(coef_df)

In [ ]:
px.scatter(x=y_test, y=y_pred, title="Actual vs Predicted Sales")


In [ ]:
regressor = Ridge(alpha=0.005)
regressor.fit(x_train, y_train)
scores = cross_val_score(regressor, x_train, y_train, cv=3)
print("Cross-validation scores:", scores)
print("Average cross-validation score:", np.mean(scores))

In [ ]:
regressor = Ridge()

params = {
    "alpha": [0,0.05,0.2, 0.5, 1, 1.5, 2, 3]
}

gridsearch = GridSearchCV(regressor, param_grid = params, cv = 3) # cv : the number of folds to be used for CV
gridsearch.fit(x_train, y_train)

In [ ]:
print("Best hyperparameters : ", gridsearch.best_params_)
print("Best R2 score : ", gridsearch.best_score_)

In [ ]:
pd.DataFrame.from_dict(gridsearch.cv_results_).T

In [ ]:
regressor = Lasso()

params = {
    'alpha': [0.05,0.1,0.3,0.5,0.8, 1.4, 1.5, 1.7] 
}

gridsearch = GridSearchCV(regressor, param_grid = params, cv = 3) # cv : the number of folds to be used for CV
gridsearch.fit(x_train, y_train)

In [ ]:
pd.DataFrame.from_dict(gridsearch.cv_results_).T.iloc[4:]

In [ ]:
best_regressor = gridsearch.best_estimator_
y_test_pred = best_regressor.predict(x_test)

In [ ]:
r2_score(y_test, y_test_pred)

In [ ]:
best_feature_importance = pd.DataFrame(
    {
        'feature': preprocessor.get_feature_names_out(),
        'coef_linear': model.coef_,
        'coef_lasso': best_regressor.coef_
    }
)



In [ ]:
features_to_keep = best_feature_importance[best_feature_importance['coef_lasso'] > 0]['feature'].tolist()
features_to_keep

In [ ]:
X_reduced = x[features_to_keep]

xr_train, xr_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.2, random_state=42)

xr_train = preprocessor.fit_transform(xr_train)
xr_test = preprocessor.transform(xr_test)

In [ ]:
final_regressor = LinearRegression()

final_regressor.fit(xr_train, y_train)

print(final_regressor.score(xr_train, y_train))
print(final_regressor.score(xr_test, y_test))

In [ ]:
final_lasso = Lasso(alpha=1)

final_lasso.fit(xr_train, y_train)

print(final_lasso.score(xr_train, y_train))
print(final_lasso.score(xr_test, y_test))